# Clonamos el repositorio para obtener los dataSet

In [ ]:
!git clone https://github.com/joanby/tensorflow.git

# Damos acceso a nuestro Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Test it

In [ ]:
!ls '/content/drive/My Drive' 

# Google colab tools

In [ ]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

##Especificando la versión de TensorFlow

Ejecutando "importar tensorflow" importará la versión por defecto (actualmente 2.x). Puedes usar la 1.x ejecutando una celda con la "versión mágica de tensorflow" **antes de ejecutar "importar tensorflow".

### Si no funciona hacer el pip install


In [ ]:
#!pip install tensorflow==1.14
%tensorflow_version 1.x

# Importar Tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt

In [ ]:
session = tf.Session()

# Una red neuronal convolucional para detectar objetos

In [ ]:
import os 
import sys
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import urllib
session = tf.Session()

In [ ]:
batch_size = 128
output_every = 50
generations = 20000
eval_every = 500
image_height = 32
image_width = 32
crop_height = 24
crop_width = 24
num_channels = 3
num_targets = 10
data_folder = "cifar-10-batches-bin"

$$learning\ rate = 0.1\cdot 0.9^{\frac{x}{250}}$$

In [ ]:
learning_rate = 0.1
lr_decay = 0.9
num_gens_to_wait = 250

In [ ]:
image_vect_length = image_height*image_width*num_channels
record_length = 1+image_vect_length

### Descarga y procesamiento de CIFAR 10

In [ ]:
data_dir = '/content/tensorflow/datasets/cifar-10-temp'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
cifar10_url = "http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz"
data_file = os.path.join(data_dir, 'cifar-10-binary.tar.gz')
if not os.path.isfile(data_file):
    filepath, _ = urllib.request.urlretrieve(cifar10_url, data_file)
    tarfile.open(filepath, 'r:gz').extractall(data_dir)

In [ ]:
def read_cifar_files(filename_queue, distort_images = True):
    reader = tf.FixedLengthRecordReader(record_bytes=record_length)
    key, record_string = reader.read(filename_queue)
    #creamos fichero binario
    record_bytes = tf.decode_raw(record_string, tf.uint8)
    #extraemos la etiqueta
    image_label = tf.cast(tf.slice(record_bytes, [0], [1]), tf.int32)
    #extraemos la imagen
    image_extracted = tf.reshape(tf.slice(record_bytes, [1], [image_vect_length]), [num_channels, image_height, image_width])
    #redimensión de imagen
    reshaped_image = tf.transpose(image_extracted, [1,2,0])
    reshaped_image = tf.cast(reshaped_image, tf.float32)
    #crop aleatorio
    final_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, crop_width, crop_height)
    if distort_images:
        #flip aleatorio horizontal, cambiar brillo y contraste
        final_image = tf.image.random_flip_left_right(final_image)
        final_image = tf.image.random_brightness(final_image, max_delta=63)
        final_image = tf.image.random_contrast(final_image, lower=0.2, upper=1.8)
    #estandarización por color
    final_image = tf.image.per_image_standardization(final_image)
    return final_image, image_label

In [ ]:
def input_pipeline(batch_size, train_logical=True):
    if train_logical:
        files = [os.path.join(data_dir, data_folder, 'data_batch_{}.bin'.format(i)) for i in range(1,6)]
    else:
        files = [os.path.join(data_dir, data_folder, 'test_batch.bin')]
    
    filename_queue = tf.train.string_input_producer(files)
    image, label = read_cifar_files(filename_queue)
    
    min_after_dequeue = 1000
    capacity = min_after_dequeue+3*batch_size
    example_batch, label_batch = tf.train.shuffle_batch([image,label], batch_size, capacity, min_after_dequeue)
    return example_batch, label_batch

In [ ]:
def cifar_cnn_model(input_images, batch_size, train_logical=True):
    def truncated_normal_var(name, shape, dtype):
        return tf.get_variable(name=name, shape=shape, dtype=dtype, 
                               initializer=tf.truncated_normal_initializer(stddev=0.05))
    def zero_var(name, shape, dtype):
        return tf.get_variable(name=name, shape=shape, dtype=dtype, 
                              initializer=tf.constant_initializer(0.0))
    ## Primera capa de convolución
    with tf.variable_scope("conv1") as scope:
        ##Conv de 5x5 para 3 canales con un total de 64 nodos al final
        conv1_kernel = truncated_normal_var(name="conv_lernel1", shape=[5,5,3,64], dtype=tf.float32)
        conv1 = tf.nn.conv2d(input_images, conv1_kernel, [1,1,1,1],padding="SAME")
        conv1_bias = zero_var(name="conv_bias1", shape=[64], dtype=tf.float32)
        conv1_add_bias = tf.nn.bias_add(conv1, conv1_bias)
        ## Capa de relu
        relu_conv1 = tf.nn.relu(conv1_add_bias)
    #Max pooling
    pool1 = tf.nn.max_pool(relu_conv1, ksize=[1,3,3,1], strides=[1,2,2,1],padding="SAME", name="pool_layer1")
    
    #Normalización local
    norm1 = tf.nn.lrn(pool1, depth_radius=5, bias=2.0, alpha=1e-3, beta=0.75, name="norm1")
    
    ## Segunda capa de convolución
    with tf.variable_scope("conv2") as scope:
        ##Conv de 5x5 para 64 nodos de entrada con un total de 64 nodos al final
        conv2_kernel = truncated_normal_var(name="conv_lernel2", shape=[5,5,64,64], dtype=tf.float32)
        conv2 = tf.nn.conv2d(norm1, conv2_kernel, [1,1,1,1],padding="SAME")
        conv2_bias = zero_var(name="conv_bias2", shape=[64], dtype=tf.float32)
        conv2_add_bias = tf.nn.bias_add(conv2, conv2_bias)
        ## Capa de relu
        relu_conv2 = tf.nn.relu(conv2_add_bias)
    #Max pooling
    pool2 = tf.nn.max_pool(relu_conv2, ksize=[1,3,3,1], strides=[1,2,2,1],padding="SAME", name="pool_layer2")
    
    #Normalización local
    norm2 = tf.nn.lrn(pool2, depth_radius=5, bias=2.0, alpha=1e-3, beta=0.75, name="norm2")
    
    #Redimensionar a una matriz para poder multiplicar en las capas totalmente conectadas
    reshaped_output = tf.reshape(norm2, [batch_size,-1])
    reshaped_dim = reshaped_output.get_shape()[1].value
    
    ##Primera capa totalmente conectada
    with tf.variable_scope("full1") as scope:
        #Full connected con 384 nodos
        full_weight1 = truncated_normal_var(name="full_mult1", shape=[reshaped_dim, 384], dtype = tf.float32)
        full_bias1 = zero_var(name="full_bias1", shape=[384], dtype=tf.float32)
        full_layer1 = tf.nn.relu(tf.add(tf.matmul(reshaped_output, full_weight1), full_bias1))
    ##Segunda capa totalmente conectada
    with tf.variable_scope("full2") as scope:
        #Full connected con 192 nodos
        full_weight2 = truncated_normal_var(name="full_mult2", shape=[384, 192], dtype = tf.float32)
        full_bias2 = zero_var(name="full_bias2", shape=[192], dtype=tf.float32)
        full_layer2 = tf.nn.relu(tf.add(tf.matmul(full_layer1, full_weight2), full_bias2))
    ##Tercera capa totalmente conectada
    with tf.variable_scope("full3") as scope:
        #Full connected con una de las 10 categorías de salida
        full_weight3 = truncated_normal_var(name="full_mult3", shape=[192, num_targets], dtype = tf.float32)
        full_bias3 = zero_var(name="full_bias3", shape=[num_targets], dtype=tf.float32)
        final_output = tf.add(tf.matmul(full_layer2, full_weight3), full_bias3)
    
    return final_output

In [ ]:
def cifar_loss(logits, targets):
    targets = tf.squeeze(tf.cast(targets, tf.int32))
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=targets)
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    return cross_entropy_mean

In [ ]:
def train_step(loss_value, generation_num):
    model_learning_rate = tf.train.exponential_decay(learning_rate, generation_num, 
                                                     num_gens_to_wait, lr_decay, staircase=True)
    my_optim = tf.train.GradientDescentOptimizer(model_learning_rate)
    train_step = my_optim.minimize(loss_value)
    return train_step

In [ ]:
def accuracy_from_batches(logits, targets):
    targets = tf.squeeze(tf.cast(targets, tf.int32))
    batch_predictions = tf.cast(tf.argmax(logits,1), tf.int32)
    predicted_correctly = tf.equal(batch_predictions, targets)
    accuracy = tf.reduce_mean(tf.cast(predicted_correctly, tf.float32))
    return accuracy

### Entrenamiento

In [ ]:
images, targets = input_pipeline(batch_size, train_logical=True)
test_images, test_targets = input_pipeline(batch_size, train_logical=False)

In [ ]:
with tf.variable_scope('model_definition') as scope:
    model_output = cifar_cnn_model(images, batch_size)
    scope.reuse_variables()
    test_output = cifar_cnn_model(test_images, batch_size)

In [ ]:
loss = cifar_loss(model_output, targets)
accuracy = accuracy_from_batches(test_output, test_targets)
generation_num = tf.Variable(0, trainable=False)
train_op = train_step(loss, generation_num)

In [ ]:
init = tf.global_variables_initializer()
session.run(init)

In [ ]:
tf.train.start_queue_runners(sess=session)

In [ ]:
train_loss = []
test_acc = []
for i in range(generations):
    _, loss_value = session.run([train_op, loss])
    if (i+1) % output_every == 0:
        train_loss.append(loss_value)
        print("Paso {}, Pérdida: {:.5f}".format((i+1),loss_value))
    if (i+1) % eval_every == 0:
        [temp_acc] = session.run([accuracy])
        test_acc.append(temp_acc)
        print("--- Precisión en test: {:.2f}%. ---".format(100*temp_acc))

In [ ]:
eval_idx = range(0, generations, eval_every)
output_idx = range(0,generations, output_every)

In [ ]:
plt.plot(output_idx, train_loss, 'k-')
plt.title("Softmax Loss para cada iteración")
plt.xlabel("Iteración")
plt.ylabel("Pérdida en Softmax")
plt.show()

In [ ]:
plt.plot(eval_idx, test_acc, 'k-')
plt.title("Mejora de la precisión en cada iteración")
plt.xlabel("Iteración")
plt.ylabel("Precisión")
plt.ylim([0,1])
plt.show()